In [7]:
import os
import json
import math
import numpy as np
import pandas as pd
from skimage import io
from skimage.transform import resize

In [8]:
# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid' 
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_tain.json'  
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid.json'

In [9]:
# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
    
    return np.array(images), labels

In [10]:
# Fungsi Euclidean Distance
def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

In [11]:
# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
    
    return np.array(images), labels

In [12]:
# Muat data latih
train_data, train_labels = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels = load_data_from_json_and_images(image_test_folder, json_test_file)



In [14]:
# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    print(f"Test Image {i+1}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

Test Image 1: Predicted Label = Red, True Label = Red
Test Image 2: Predicted Label = Red, True Label = Red
Test Image 3: Predicted Label = Red, True Label = Red
Test Image 4: Predicted Label = Red, True Label = Green
Test Image 5: Predicted Label = Blue, True Label = Blue
Test Image 6: Predicted Label = Red, True Label = Green
Test Image 7: Predicted Label = Red, True Label = Red
Test Image 8: Predicted Label = Red, True Label = Red
Test Image 9: Predicted Label = Blue, True Label = White
Test Image 10: Predicted Label = Green, True Label = Green
Test Image 11: Predicted Label = Red, True Label = Blue
Test Image 12: Predicted Label = Red, True Label = Green
Test Image 13: Predicted Label = Red, True Label = Green
Test Image 14: Predicted Label = Red, True Label = Red
Test Image 15: Predicted Label = Blue, True Label = Blue
Test Image 16: Predicted Label = Red, True Label = Green
Test Image 17: Predicted Label = Black, True Label = Blue
Test Image 18: Predicted Label = Red, True Label 